In [2]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [3]:
!pip install -q cassio datasets langchain openai tiktoken


In [4]:
 # LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [6]:
from PyPDF2 import PdfReader

In [18]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:zfboQzHnPxSbSYmEeyMZLysv:a72d966b1bbeb00485b364a40fb33278258246f9ecb1011e2732e1628613b896" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "aa0f9704-f528-4d33-bb0a-6e4f694ca992" # enter your Database ID

OPENAI_API_KEY = "sk-None-G2VcIn6W9aOBblloR5S0T3BlbkFJCZ1VggNrqspHD5RdLObK" # enter your OpenAI key

In [19]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('PDF_test.pdf')

In [20]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [21]:
raw_text

'1 \n A&P   \nby John Updike - 1962  \nIn walks these three girls in nothing but bathing suits. I\'m in the third check -out \nslot, with my back to the door, so I don\'t see them until they\'re over by the bread. \nThe one that caught my eye first was the one in the plaid gr een two -piece. She \nwas a chunky kid, with a good tan and a sweet broad soft -looking can with those \ntwo crescents of white just under it, where the sun never seems to hit, at the top \nof the backs of her legs. I stood there with my hand on a box of HiHo crac kers \ntrying to remember if I rang it up or not. I ring it up again and the customer starts \ngiving me hell. She\'s one of these cash -register -watchers, a witch about fifty with \nrouge on her cheekbones and no eyebrows, and I know it made her day to trip \nme up . She\'d been watching cash registers forty years and probably never seen a \nmistake before.  \nBy the time I got her feathers smoothed and her goodies into a bag -- she gives \nme a little

In [22]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [23]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

<ipython-input-23-9c33d76eff6b>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)
<ipython-input-23-9c33d76eff6b>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [24]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [25]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [26]:
texts[:50]

["1 \n A&P   \nby John Updike - 1962  \nIn walks these three girls in nothing but bathing suits. I'm in the third check -out \nslot, with my back to the door, so I don't see them until they're over by the bread. \nThe one that caught my eye first was the one in the plaid gr een two -piece. She \nwas a chunky kid, with a good tan and a sweet broad soft -looking can with those \ntwo crescents of white just under it, where the sun never seems to hit, at the top \nof the backs of her legs. I stood there with my hand on a box of HiHo crac kers \ntrying to remember if I rang it up or not. I ring it up again and the customer starts \ngiving me hell. She's one of these cash -register -watchers, a witch about fifty with \nrouge on her cheekbones and no eyebrows, and I know it made her day to trip",
 "giving me hell. She's one of these cash -register -watchers, a witch about fifty with \nrouge on her cheekbones and no eyebrows, and I know it made her day to trip \nme up . She'd been watching cas

In [27]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [28]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): What had the old man in "A Clean, Well-Lighted Place" recently tried to do?

QUESTION: "What had the old man in "A Clean, Well-Lighted Place" recently tried to do?"


ANSWER: "The old man had recently tried to commit suicide."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9199] "windows, over the bags of peat moss and aluminum lawn furniture stacked on 
the pave ..."
    [0.9198] "windows, over the bags of peat moss and aluminum lawn furniture stacked on 
the pave ..."
    [0.9174] "the street was dusty; but at night the dew settled the dust and the old man 
liked t ..."
    [0.9174] "the street was dusty; but at night the dew settled the dust and the old man 
liked t ..."

What's your next question (or type 'quit' to exit): quit
